In [ ]:
from __future__ import print_function

In [ ]:
!unzip '/content/drive/MyDrive/Colab_Notebooks/spotify_million_playlist_dataset.zip' -d  mydata

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import codecs
import numpy
import pandas as pd
from pandas import json_normalize
import os
import json
import time
import numpy
import pandas as pd
import sys
from scipy.sparse import coo_matrix

!pip install implicit
!pip install annoy

from implicit.als import AlternatingLeastSquares
from implicit.approximate_als import AnnoyAlternatingLeastSquares
from implicit.bpr import BayesianPersonalizedRanking

#Output file to add the recommendatio for the test dataset.
outputfile = '//content/drive/MyDrive/Somedata/my_results_merge.csv'

In [ ]:
def get_mpd_data(path):

    # creating empty df for the pid and track uri
    tracks_df = pd.DataFrame(columns=['pid','track_uri'])

    #path_json = path
    #Fetch all the mpd slice data from the folder.
    all_files = [slice_json for slice_json in os.listdir(path) if slice_json.endswith('.json')]
    all_files.sort()

    # Get all the tracks uri and playlistID
    for index, js in enumerate(all_files):
      with open(os.path.join(path, js)) as all_files:
          json_file = json.load(all_files)

          # Fetch tracks list for the all  pids.
          tracks = pd.json_normalize(json_file['playlists'], record_path='tracks', meta=['pid'])

          # Add only pid and track_uri into the DF
          tracks = tracks[['pid', 'track_uri']]
          tracks_df = tracks_df.append(tracks)

    #Writting Tracks list into the CSV files.
    tracks_df.to_csv('/content/drive/MyDrive/Somedata/mpd.csv', index=False)

In [ ]:
get_mpd_data('/content/drive/MyDrive/Somedata')

In [ ]:
#Read the Challenge Dataset for recommending the task.
def read_challenge_dataset(path):

    #Read the challenge dataset
    challenge_df = pd.read_json(path +'/challenge_set.json')

    #Normalize the json data
    #fetch the track data along pif and num_sample colums.
    ch_tracks = json_normalize(challenge_df['playlists'], record_path='tracks',
                            meta=['pid','num_samples'], errors='ignore')

    #Considering only the pid and track_uri
    ch_tracks = ch_tracks[['pid','track_uri']]
    ch_df_ids = ch_tracks.copy()

    #Calling function to generate seed data for the challenge dataset playlist with no columns.
    final_challenge_df = generating_seed_data(challenge_df, ch_df_ids, ch_tracks)

    #Reorder the column names.
    final_challenge_df = final_challenge_df.reindex(columns=['pid', 'track_uri', 'track_count'])
    
    # Set track count =1 to indicate the presence of song in the playlist.
    final_challenge_df['track_count']=1

    return final_challenge_df

In [ ]:
#Get the title similary csv with recommended track.
cold_start_recommendation = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/CMPE256/Project/data/Title_Similarty_Data/title_popularity_recs.csv')

In [ ]:
cold_start_recommendation.shape

(747, 501)

In [ ]:
# Using this method to normalize the title and compare with the title similarity  df.
import re
def normalize_name(name):
    name = name.lower()
    name = re.sub(r"[.,\/#!$%\^\*;:{}=\_`~()@]", ' ', name)
    name = re.sub(r'\s+', ' ', name).strip()
    return name

In [ ]:
# Getting tracks for challenge playlist with only titles.
# if recommendation is not found using the generating random seed tracks for the such playlists.

#first line of the submission file.
first_line = 'team_info,Team5,main,Bijayanismishta@gmail.com'

def generating_seed_data(ch_df, challenge_df, tracks_ch):

    # Openning the output file to write the recommendation from the title similarity matrix.
    with codecs.open(outputfile, "w") as o:
      o.write("%s \n" %(first_line))
      o.write("\n")
      for i, r in ch_df.iterrows():
        #Checking for playlist with no seed tracks.
          if (ch_df['playlists'][i]['num_samples']==0):
              name = ch_df['playlists'][i]['name']
              pid = ch_df['playlists'][i]['pid']
              name = normalize_name(name)

              # Extracting the recommendation based on playlist name.
              row = cold_start_recommendation[cold_start_recommendation['Names'] == name]
              row = row.dropna(axis=1, how='all')

              # Checking if the recommendation list has 500 songs ad writting the recommended songs in the output files.
              if row.shape[1] == 501:
                df_list = row.values.tolist()
                df_list = df_list.pop()[1:]
                o.write("%s" %(pid) + ',')
                recs = ','.join(map(str, df_list))
                o.write(recs)
                o.write("\n")
                o.write("\n")
                recs = ['']
              else:
                #For playlist no matching title or less than 500 song recommendation, adding random songs as seed tracks.
                try:
                 # through random sampling getting n songs, adding pid and appending to final challenge_df
                  n = 10
                  sample_instance = tracks_ch.track_uri.sample(n)
                  samples_df = pd.DataFrame(sample_instance)
                  samples_df['pid']=ch_df['playlists'][i]['pid']
                  challenge_df = challenge_df.append(samples_df, sort=False)
                except Error as e:
                  print("Error occured :", e)
    return challenge_df

In [ ]:
#Method to remove the less frequest songs.
def removing_less_frequent_songs():


    #Getting mpd slice data.
    data_df = pd.read_csv('/content/drive/MyDrive/Somedata/mpd.csv')

    #Adding new Column as track_count and setting it to one to indicate song exits in the paylist.
    data_df['track_count']=1

    # Creating a copy of the mpd data to get count of the data.
    count_df = data_df.copy()
    print(data_df)

    # get the count all repeated songs in the data.
    count_df = count_df.groupby(['track_uri'])['track_count'].sum().reset_index()
    print(count_df)
    
    # removing less frequent songs that repeats less than 200 times in the mpd.
    data_cleaned = count_df[count_df['track_count']> 200]

    # By inner join getting only the dats of songs that occurs more 200 times in the dataset.
    data_merged = pd.merge(data_df, data_cleaned, how='inner', on=['track_uri'])
    
    # renaming column
    data_merged.rename(columns={"track_count_x": "track_count"},inplace=True)
    data_merged = data_merged[['pid','track_uri','track_count']]

    print("data merge : ", data_merged.head())
    return data_merged


In [ ]:
# Model included as part of the recommendation system
MODELS = {"als_model":  AlternatingLeastSquares,
          "annoy_als_model": AnnoyAlternatingLeastSquares,
          "bpr_model": BayesianPersonalizedRanking
          }

In [ ]:
def fetch_model_details(model_name):
    model = MODELS.get(model_name)

    #If model_name is invalid throw error.
    if not model:
        raise ValueError("Invalid model name '%s'" % model)

    # Set hyperParameters for the models
    if issubclass(model, AlternatingLeastSquares):
        params = {'factors': 200, 'iterations': 40, 'dtype': numpy.float32, 'use_gpu': True, 'calculate_training_loss':True, 'use_cg':True, 'use_native':True}
    elif model_name == "bpr":
        params = {'factors': 20, 'use_gpu': False}
    else:
        params = {}

    return model(**params)

In [ ]:
#Read the merged dataset and return the sparse matrix and tuple for pid, track_uri and track count.
def read_data(filename):
   
    # Read the final_data i.e merge datset of training and challenge data set.
    data = pd.read_csv(filename, dtype={"pid": 'category', "track_uri": 'category', "track_count": int})
      
    # map each playlist and track uri  to a unique numeric value
    data['track_uri'] = data['track_uri'].astype('category')
    data['pid'] = data['pid'].astype('category')
    data['track_count'] = data['track_count'].astype(int)

    # create a coordinate matrix for pid, track_uri and track count.
    matrix = coo_matrix((data['track_count'].astype(numpy.float32),
                       (data['track_uri'].cat.codes.copy(),
                        data['pid'].cat.codes.copy())))

    return data, matrix

In [ ]:
#Renerate recommendatio track for each pid.
def generate_recommendations(input_filename, output_filename, model_name):

    #Read the merge data and fetch the sparse matrix and pid, track_uri and track count data.
    df, track_count = read_data(input_filename)

    #Create the base model to train.
    model = fetch_model_details(model_name)

    # setting the approximate similar_item false for the ALS model.
    if issubclass(model.__class__, AlternatingLeastSquares):
        model.approximate_similar_items = False

    # Get the csr matrix.
    track_count = track_count.tocsr()

    #Train the model on the csr matrix 
    model.fit(track_count)

    # get the recommendations tracks from the trained model.
    recs = ['']
    
    tracks = dict(enumerate(df['track_uri'].cat.categories))
    pid_track_counts = track_count.T.tocsr()

    #Write the recommended tracks into the CSV files.
    with codecs.open(output_filename, "a") as o:
        for playlist_id, pid in enumerate(df['pid'].cat.categories):
          # Writting csv files for all the playlist that has pid > 1000000, i.e. the challenge dataset.
          if int(pid)>=1000000:
            for track_id, score in model.recommend(playlist_id, pid_track_counts, N=500):
                    recs.append(tracks[track_id])
            o.write("%s" %(pid))
            recs = ','.join(map(str, recs))
            o.write(recs)
            o.write("\n")
            o.write("\n")

            recs = ['']
    print("Done!!")

In [ ]:
#Select one the model als_model, bpr_model or annoy_model
model = 'als_model'


# Reading  the challenge data set.
challenge_df = read_challenge_dataset('/content/drive/MyDrive/Colab_Notebooks/CMPE256/Project/challenge_set')

#Remove the less frequent songs from the track list
mpd = removing_less_frequent_songs()

#Merging mpd data with challenge_sets to train the models.
final_data = mpd.append(challenge_df)

print("final data ", final_data)

#Writting the files into the csv files.
final_data.to_csv('/content/drive/MyDrive/Somedata/mpd_ch.csv', index=False)


#Mathod to calculate the recommendation list with als model
generate_recommendations('/content/drive/MyDrive/Somedata/mpd_ch.csv','/content/drive/MyDrive/Somedata/my_results_merge.csv', model)


          pid                             track_uri  track_count
0           0  spotify:track:0UaMYEvWZi0ZqiDOoHU3YI            1
1           0  spotify:track:6I9VzXrHxO9rA9A5euc8Ak            1
2           0  spotify:track:0WqIKmW4BTrj3eJFmnCKMv            1
3           0  spotify:track:1AWQoqb9bSvzTjaLralEkT            1
4           0  spotify:track:1lzr43nnXAijIGYnCT8M8H            1
...       ...                                   ...          ...
265707  11999  spotify:track:2ovm5VZJ36s3HKF8nkcIZI            1
265708  11999  spotify:track:2IqCOKEx7mzSp4BTxkdFIV            1
265709  11999  spotify:track:4Dw02sVUfUA67l3fZ9FoKs            1
265710  11999  spotify:track:4qhkS0lRL2GgDxpuIr2Mbb            1
265711  11999  spotify:track:2FQ5y4bYmq2h974oUBxoZH            1

[265712 rows x 3 columns]
                                  track_uri  track_count
0      spotify:track:000JBgYWfJQqdFaRqu2n3f            1
1      spotify:track:000mA0etY38nKdvf1N04af            1
2      spotify:track:0